1. Import Library

pandas (pd) → untuk membaca dataset dan mengolah data dalam bentuk tabel (DataFrame).

numpy (np) → digunakan untuk operasi numerik, termasuk membuat array label unik.

train_test_split → memisahkan dataset menjadi data training dan testing.

StandardScaler → menstandarisasi fitur agar seluruh nilai berada dalam skala yang sama (sangat penting untuk KNN).

LabelEncoder → mengubah data kategori (string) menjadi angka.

KNeighborsClassifier → algoritma KNN untuk klasifikasi.

accuracy_score & classification_report → mengukur performa model.

joblib → menyimpan model, scaler, dan encoder dalam satu file .pkl agar bisa dipakai ulang.

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

2. LOAD DATASET

membaca file datasets.csv.

header=3 artinya baris ke-4 (index 3) dianggap sebagai header/kolom.

Jika file tidak ditemukan → program berhenti dengan pesan error.

In [26]:
try:
    df = pd.read_csv('datasets.csv', header=3)
except FileNotFoundError:
    print("Error: File 'datasets.csv' tidak ditemukan.")
    exit()

3. DATA CLEANING

Menentukan daftar kolom yang tidak digunakan.

existing_cols memastikan hanya kolom yang benar-benar ada yang dihapus (menghindari error).

df_clean adalah dataset bersih yang siap digunakan.

In [27]:
drop_cols = ['NIM', 'Nama', 'SemesterDropout', 'JenisKelamin']
existing_cols = [col for col in drop_cols if col in df.columns]
df_clean = df.drop(columns=existing_cols)

Pisahkan Fitur dan Target

X → semua fitur input.

y → kolom output target (label kelas).

In [28]:
X = df_clean.drop(columns=['Target'])
y = df_clean['Target']

4. ENCODING Fitur kategori

Mendeteksi semua kolom bertipe object (string).

Setiap kolom string harus diubah menjadi angka → dibutuhkan untuk KNN.

Encoder untuk masing-masing kolom disimpan dalam dictionary label_encoders supaya bisa dipakai lagi pada prediksi baru.

In [29]:
label_encoders = {}
cat_cols = X.select_dtypes(include=['object']).columns

print("Sedang memproses data kategori...")
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

Sedang memproses data kategori...


Encode Target

Kolom target juga harus berupa angka.

Disimpan ke target_le agar nanti laporan klasifikasi tetap menampilkan nama kelas asli.

In [30]:
target_le = LabelEncoder()
y = target_le.fit_transform(y)

5. SCALING 

Wajib karena KNN sensitif terhadap jarak antar titik.
Jika skala fitur berbeda jauh (misal usia 20–30 vs gaji 1 juta–10 juta), model jadi bias.

StandardScaler membuat fitur memiliki:

a) mean = 0

b) std = 1

Sehingga semua fitur berada dalam skala yang sama.

In [31]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

6. SPLIT DATA

Memisahkan data → 80% train, 20% test.

stratify=y memastikan distribusi kelas di train dan test tetap seimbang.

random_state=42 untuk memastikan hasil tetap sama setiap dijalankan.

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

7. TRAINING MODEL KNN

n_neighbors=3 → jumlah tetangga = 3.

metric='euclidean' → menggunakan hitungan jarak Euclidean.

.fit() → melatih model dengan data training.

In [33]:
knn = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn.fit(X_train, y_train)

KNeighborsClassifier(metric='euclidean', n_neighbors=3)

8. EVALUASI MODEL

predict() membuat prediksi pada data testing.

accuracy_score() menghitung akurasi model.

In [34]:
y_pred = knn.predict(X_test)
print(f"\nAkurasi Model: {accuracy_score(y_test, y_pred) * 100:.2f}%")


Akurasi Model: 100.00%


Classification Report

Untuk mencegah error jika suatu kelas tidak muncul di y_test.

Menampilkan:

1) precision

2) recall

3) f1-score

4) support

Dan nama kelas yang ditampilkan tetap sesuai label aslinya.

In [35]:
unique_labels = np.arange(len(target_le.classes_))
print("\nLaporan Klasifikasi:\n", classification_report(
    y_test, y_pred, labels=unique_labels, target_names=target_le.classes_
))


Laporan Klasifikasi:
                    precision    recall  f1-score   support

          Dropout       0.00      0.00      0.00         0
Lulus Tepat Waktu       1.00      1.00      1.00         9
  Lulus Terlambat       1.00      1.00      1.00         2

         accuracy                           1.00        11
        macro avg       0.67      0.67      0.67        11
     weighted avg       1.00      1.00      1.00        11



/Users/rifqizvar/VSCode/SEMESTER_5/DataMining/PythonNotebook/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rifqizvar/VSCode/SEMESTER_5/DataMining/PythonNotebook/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/rifqizvar/VSCode/SEMESTER_5/DataMining/PythonNotebook/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to

9. SIMPAN MODEL + ARTIFACT

Semua komponen penting disimpan dalam 1 file .pkl:

1) model KNN yang sudah dilatih

2) scaler (untuk standarisasi input baru)

3) encoder tiap fitur kategori

4) encoder target

5) nama fitur (untuk prediksi)

6) daftar kolom kategori

Agar saat membuat GUI (misalnya Streamlit) atau sistem prediksi real-time, semua proses preprocessing bisa dilakukan sama persis seperti saat training.

In [36]:
artifacts = {
    'model': knn,
    'scaler': scaler,
    'encoders': label_encoders,
    'target_encoder': target_le,
    'feature_names': X.columns.tolist(),
    'cat_cols': cat_cols.tolist()
}
joblib.dump(artifacts, 'student_prediction_artifacts.pkl')

['student_prediction_artifacts.pkl']